In [19]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Création du DataFrame
data = {
    'DBMS': ['PostgreSQL', 'MySQL', 'SQLite', 'Microsoft SQL Server', 'MongoDB', 
             'Redis', 'MariaDB', 'Elasticsearch', 'DynamoDB', 'Oracle'],
    'Popularité': [49.09, 40.59, 30.17, 27.34, 25.66, 23.25, 17.69, 15.33, 10.31, 10.06],
    'Type': ['SQL', 'SQL', 'SQL', 'SQL', 'NoSQL', 'NoSQL', 'SQL', 'NoSQL', 'NoSQL', 'SQL']
}

df = pd.DataFrame(data)

# Séparation des données par type
df_sql = df[df['Type'] == 'SQL'].sort_values('Popularité', ascending=True)
df_nosql = df[df['Type'] == 'NoSQL'].sort_values('Popularité', ascending=True)

# Création des sous-graphiques
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Bases de données SQL', 'Bases de données NoSQL'),
    specs=[[{"type": "bar"}, {"type": "bar"}]],
    column_widths=[0.5, 0.5]
)

# Ajout des barres pour SQL
fig.add_trace(
    go.Bar(
        x=df_sql['Popularité'],
        y=df_sql['DBMS'],
        orientation='h',
        text=df_sql['Popularité'].round(1).astype(str) + '%',
        textposition='outside',
        marker_color='rgb(70,70,70)',
        name='SQL',
        showlegend=False
    ),
    row=1, col=1
)

# Ajout des barres pour NoSQL
fig.add_trace(
    go.Bar(
        x=df_nosql['Popularité'],
        y=df_nosql['DBMS'],
        orientation='h',
        text=df_nosql['Popularité'].round(1).astype(str) + '%',
        textposition='outside',
        marker_color='rgb(180,180,180)',
        name='NoSQL',
        showlegend=False
    ),
    row=1, col=2
)

# Mise à jour du layout
fig.update_layout(
    title=dict(
        text='<b>Top 10 des SGBD SQL vs NoSQL</b><br><sup>Stack Overflow Developer Survey 2023</sup>',
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=20)
    ),
    height=500,
    width=1200,
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Mise à jour des axes
for i in [1, 2]:
    fig.update_xaxes(
        title="Taux d'adoption (%)",
        showgrid=False,  # Retire la grille
        zeroline=False,  # Retire la ligne du zéro
        range=[0, max(df['Popularité']) * 1.15],
        row=1, col=i
    )
    fig.update_yaxes(
        title="",
        showgrid=False,  # Retire la grille
        zeroline=False,  # Retire la ligne du zéro
        row=1, col=i
    )

# Ajout de la source avec le lien
fig.add_annotation(
    text="Source : https://survey.stackoverflow.co/2023/",
    xref="paper",
    yref="paper",
    x=0,
    y=-0.22,
    showarrow=False,
    font=dict(size=10, color='gray'),
    align="left"
)

# Affichage
fig.show()

## SQL databases

### PostgreSQL

In [ ]:
```python
import os
import sys
from loguru import logger
import psycopg2


def connect_to_pg_db():
    """Connect to our PostgreSQL database"""
    
    conn = None
    try:
        logger.info('Connecting')
        conn = psycopg2.connect(host=os.environ['DB_PATH'],
                                database=os.environ['DB_NAME'],
                                user=os.environ['DB_USERNAME'],
                                password=os.environ['DB_PASSWORD'])
    except (Exception, psycopg2.DatabaseError) as error:
        logger.error(error)
        sys.exit(1)
    
    logger.info('Connection successful!')
    
    return conn

In [ ]:
from configparser import ConfigParser

def config(filename='database.ini', section='postgresql'):
    # Create a parser
    parser = ConfigParser()
    # Read the configuration file
    parser.read(filename)
    # Get the information from the postgresql section
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in {1}'.format(section, filename))
    
    return db

In [ ]:
def execute_query(query):  
    """Query data from the vendors table"""  
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(query)
        logger.info("The number of parts: %s", cur.rowcount)
        row = cur.fetchone()
        while row is not None:
            logger.info(row)
            row = cur.fetchone()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        logger.error(error)
    finally:
        if conn is not None:
            conn.close()


In [ ]:
import pandas as pd

def sql_to_dataframe(conn, query, column_names):
	"""Import data from a PostgreSQL database using a SELECT query"""
	cursor = conn.cursor()
	try:      
		cursor.execute(query)  
	except (Exception, psycopg2.DatabaseError) as error:
		print("Error: %s" % error)   
		cursor.close()   
		return 1   # The execute returns a list of tuples:   
	tuples_list = cursor.fetchall()   
	cursor.close()   # Now we need to transform the list into a pandas DataFrame:   
	df = pd.DataFrame(tuples_list, columns=column_names)   
	return df

### SQLAlchemy



In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Function to create an engine and perform the query
def execute_query(query):
  '''Create an engine and execute a query, with returned results as a dataframe.'''

  # Building the connection string
  host = os.environ['DB_PATH']
  database = os.environ['DB_NAME']
  port = os.environ['DB_PORT']
  user = os.environ['DB_USER']
  password = os.environ['DB_PASSWORD']
  
  conn_str = f'postgresql://{user}:{password}@{host}:{port}/{database}' 

  # Creating the engine
  engine = create_engine(conn_str)
  
  # Opening a connection and executing the query
  with engine.connect() as connection:
      result = connection.execute(query)
      df = pd.DataFrame(result.fetchall(), columns=result.keys())
      
  # Return the dataframe
  return df

# Example of usage:
df_result = execute_query("SELECT * FROM matable")

print(df_result)

### MySQL

In [ ]:
import mysql.connector
from mysql.connector import errorcode


def query_mysql(db_user, db_password, db_host, db_database, query):
    '''
    This function is for connecting to a MySQL database,
    executing a query, and returning the results in a pandas DataFrame.
    '''

    try:
        cnx = mysql.connector.connect(user=db_user,
                                      password=db_password,
                                      host=db_host,
                                      database=db_database)
        cursor = cnx.cursor()

        # Query execution and fetching the result into result_set
        cursor.execute(query)
        result_set = cursor.fetchall()

        # Fetch the column names from the cursor description
        column_names = [column[0] for column in cursor.description]

        # Closing the connection
        cursor.close()
        cnx.close()

        # Constructing the DataFrame
        return pd.DataFrame(result_set, columns=column_names)

    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

### SQLite

In [ ]:
import sqlite3
import pandas as pd

def query_sqlite(database, query):
    '''
    This function is for connecting to a SQLite database,
    executing a query, and returning the results in a pandas DataFrame.
    '''

    try:
        # Start a connection with the SQLite database
        conn = sqlite3.connect(database)

        # Execute the query and fetch results into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the connection
        conn.close()

        return df

    except sqlite3.Error as err:
        print(str(err))
        return None

# Example usage:
df = query_sqlite('mydatabase.db', 'SELECT * FROM mytable')
print(df)